In [1]:
from surprise import KNNWithMeans, KNNBasic, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

In [2]:
import pandas as pd
unames = ['userId', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('users.dat', sep='::', 
                      header=None, names=unames, engine='python')
rnames = ['userId', 'movieId', 'rating', 'timestamp']
ratings = pd.read_table('ratings.dat', sep='::', header=None, names=rnames, engine='python')

mnames = ['movieId', 'title', 'genres']
movies = pd.read_table('movies.dat', sep='::', header=None, names=mnames, engine='python')

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [6]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,5.0,978824268.0
1,1,Toy Story (1995),Animation|Children's|Comedy,6.0,4.0,978237008.0
2,1,Toy Story (1995),Animation|Children's|Comedy,8.0,4.0,978233496.0
3,1,Toy Story (1995),Animation|Children's|Comedy,9.0,5.0,978225952.0
4,1,Toy Story (1995),Animation|Children's|Comedy,10.0,5.0,978226474.0


In [7]:
dataset_0 = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [8]:
dataset_0.head()

,uid,iid,rating
0,1.0,Toy Story (1995),5.0
1,6.0,Toy Story (1995),4.0
2,8.0,Toy Story (1995),4.0
3,9.0,Toy Story (1995),5.0
4,10.0,Toy Story (1995),5.0


In [9]:
from surprise.model_selection.validation import cross_validate

In [29]:
n = 0.1
dataset = dataset_0.sample(frac=1)[:int(n*len(dataset_0))].reset_index(drop=True)

reader = Reader(rating_scale=(dataset.rating.min(), dataset.rating.max()))
data = Dataset.load_from_df(dataset, reader)

int(n*len(dataset_0))

100020

In [30]:
algo = KNNBaseline(k=40, min_k=20, sim_options={'name': 'pearson_baseline', 'user_based': True})
result = cross_validate(algo, data, ['rmse'], n_jobs = -1, verbose = True)

Evaluating RMSE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9540  0.9435  0.9468  0.9462  0.9487  0.9478  0.0035  
Fit time          3.67    4.11    3.98    3.85    3.58    3.84    0.19    
Test time         2.54    2.38    2.31    2.09    1.88    2.24    0.23    
